In [1]:
using Pkg
Pkg.activate(".");
Pkg.status();

  Activating new environment at `C:\Users\User\Documents\Graduate Files\Physics 215\Physics-215-Julia-Codes\Session 03\Project.toml`


      Status `C:\Users\User\Documents\Graduate Files\Physics 215\Physics-215-Julia-Codes\Session 03\Project.toml` (empty project)


# KR1 ```Number``` type hierarchy in Julia

For the start of the session, we look at the hierarchy of types in Julia under ```Number```. As was shown in the session slides, there are 514 variable data types in Julia, which is a lot! In the field of physics, the most important data type is ```Number```, as in physics, we deal with very many types of numbers. So it is instructive to look at the different data types of ```Number```s that are available to us in Julia.

To make it easier to see the ```Number``` type hierarchy in Julia, let us make use of the function shown to give us the ```subtype``` tree under ```Number```. The code used came from [this website](https://julia.quantecon.org/more_julia/generic_programming.html).

In [2]:
#  from https://github.com/JuliaLang/julia/issues/24741
function subtypetree(t, level=1, indent=4)
        if level == 1
            println(t)
        end
        for s in subtypes(t)
            println(join(fill(" ", level * indent)) * string(s))  # print type
            subtypetree(s, level+1, indent)  # recursively print the next type, indenting
        end
end;

With this code, when we now look at the ```subtypetree``` corresponding to the ```Number``` data type, we get.

In [3]:
subtypetree(Number)

Number
    Complex
    Real
        AbstractFloat
            BigFloat
            Float16
            Float32
            Float64
        AbstractIrrational
            Irrational
        Integer
            Bool
            Signed
                BigInt
                Int128
                Int16
                Int32
                Int64
                Int8
            Unsigned
                UInt128
                UInt16
                UInt32
                UInt64
                UInt8
        Rational


From the ```subtypetree``` of ```Number```, we see that there are two large subtypes -- ```Complex```, and ```Real```, and that ```Real``` data type has subtypes while ```Complex``` doesn't. This makes sense since the hallmark of a complex number is the addition of $\mathrm{i}=\sqrt{-1}$, and in [Julia](https://docs.julialang.org/en/v1/manual/complex-and-rational-numbers/#Complex-Numbers), this is done with the addition of ```im``` to the number as shown.

In [4]:
typeof(3 + 2im)

Complex{Int64}

Now we move to the ```Real``` subtype, since it has more subtypes. Out of the subtypes, of particular interest to us is the ```Rational``` and ```AbstractIrrational``` subtypes.

Apparently, Julia can handle [```Rational``` numbers](https://docs.julialang.org/en/v1/manual/complex-and-rational-numbers/#Rational-Numbers), mathematically defined as the ratio of two integers. We can define rational numbers through the use of ```//``` symbol, where the number preceding it is the numerator while the number succeeding it is the denominator.

In [5]:
typeof(22 // 7)

Rational{Int64}

This is a really nice feature as it makes Julia an easier language to do symbolic mathematics with. Python can do the same with the [```fractions``` module](https://docs.python.org/3/library/fractions.html), but to define a fraction, you have to make use of the ```Fraction()``` operator, which makes your code look a little messy, so it is nice that Julia makes it easier for us!

On the other hand, we have an ```Irrational``` number subtype, which apparently is the subtype where all the [mathematical constants](https://github.com/JuliaLang/julia/blob/master/base/mathconstants.jl) are stored in. It is nice that Julia has a specific data type to store its constants.

In [6]:
print(ℯ^1)
typeof(ℯ)

2.718281828459045

Irrational{:ℯ}

where ```ℯ``` is the Euler constant as shown.

At this point, we now go to the usual subtypes when dealing with ```Number```s -- ```AbstractFloat```, and ```Integer```.

As expected, there are different subtypes of ```AbstractFloat``` and ```Integer```, depending on the number of bits are using to store the number into the computer's memory -- 8 bits, 16 bits, 32 bits, 62 bits, and 128 bits for ```Integers```, and 16 bits, 32 bits, and 64 bits for ```AbstractFloat```. As seen in the subtype tree, we also see that there is a ```BigInt``` and ```BigFloat```. As we saw in Session 2, [```BigInt```](https://docs.julialang.org/en/v1/base/numbers/#Base.GMP.BigInt) is a datatype that handles arbitrarily large integers. And in the same vein, [```BigFloat```](https://docs.julialang.org/en/v1/base/numbers/#Base.MPFR.BigFloat) is a datatype that handles arbitarily large floating point numbers.

After going down the tree using ```subtype```, we note that we can go the other way around using ```supertype```. In the code shown, we look at the supertype chain starting from ```BigFloat```.

In [7]:
data_type = BigFloat

while data_type != Any
    println(data_type)
    data_type = supertype(data_type)
end

BigFloat
AbstractFloat
Real
Number


# KR2 Constructing ```struct```s

Reading about ```struct```s, the first impression I got was that it acted like a class -- it was an object that you could use to store data referring to a single object, like a simple point mass. Normally, when we define a point mass, we have to define its position, velocity, acceleration, and mass individually. With the help of a ```struct```, we can compile all of these properties into one ```struct``` named ```PointMass``` for example, so that when we want to define a point mass, we can just call store the values of its properties into one data type,  ```PointMass```.

With ```struct```s, like ```class```es, you can put functions inside them so that once you have the basic properties, you can get back a derived property that you are most likely going to use later on in your program. However, apparently this is [not necessarily true](https://www.fluentcpp.com/2017/06/13/the-real-difference-between-struct-class/) all the time. While you can do pretty much the same basic things with ```struct```s as you would with ```class```es, there are some limitations. In the raw sense, ```struct```s are static variables, which mean you cannot change their values once you initialize them.

To give an example, let us define our own ```struct``` influenced by music.

In [8]:
struct InstrumentTimbre
    amplitude_color::Vector{Float64}
    frequency_color::Vector{Int16}
end

Our struct here is used to store the timbre of a musical instrument. Every musical instrument has its own timbre, i.e. distinct characteristic sound, that can be measured by looking at the Fourier transform of the sound it makes. So to mimic it, we store the frequency/harmonics it produces, and their corresponding amplitudes. Further, we define amplitudes to be a floating point number, while frequencies to be only integers from 1 Hz to 32767 Hz, which is more than enough for us.

Another thing I noticed while making this is that once you defined the struct, you can no longer modify it, so let's say I defined my struct to take in ```Vectors```, but decided that one should have a vector of ```Float64``` numbers while the other, ```Int16```. To do so, I have to restart the kernel and run it because after running it once and naively modifying it, Julia will throw an exception saying I have an invalid redefinition of my struct. One extra thing to take note of!

Ok, let's try to use our newly made ```struct```. Let us define an imaginary instrument with amplitude color of ```amp``` and frequency color of ```freq```, and define a variable ```theo_inst``` that has these properties.

In [21]:
amp = rand(32767)
freq = 1:32767

theo_inst = InstrumentTimbre(amp, freq)

InstrumentTimbre([0.19257820508258727, 0.3974622563099939, 0.3489329646540975, 0.7219512358278735, 0.836661777671494, 0.6855430337924562, 0.4859501131164805, 0.7817197807734115, 0.3499584001605385, 0.7042007126296368  …  0.6686687554434465, 0.21659950785945403, 0.510355247326874, 0.7973362444125665, 0.7126537559906525, 0.5015292745510369, 0.28011677993608086, 0.38679950715589384, 0.8686218098999308, 0.42377222037496765], Int16[1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  32758, 32759, 32760, 32761, 32762, 32763, 32764, 32765, 32766, 32767])

So that when we want to retrieve the properties of this instrument such as the ```amplitude color```, we can simply call.

In [25]:
theo_inst.amplitude_color

32767-element Vector{Float64}:
 0.19257820508258727
 0.3974622563099939
 0.3489329646540975
 0.7219512358278735
 0.836661777671494
 0.6855430337924562
 0.4859501131164805
 0.7817197807734115
 0.3499584001605385
 0.7042007126296368
 0.808802546082102
 0.3570620147420649
 0.22776252425003674
 ⋮
 0.08063480505388121
 0.3126471159795352
 0.6686687554434465
 0.21659950785945403
 0.510355247326874
 0.7973362444125665
 0.7126537559906525
 0.5015292745510369
 0.28011677993608086
 0.38679950715589384
 0.8686218098999308
 0.42377222037496765

Like classes, we can define multiple theoretical instruments using our ```struct```, however we cannot modify the properties of these theoretical instruments once they have been instantiated. For example, let us try to change the ```amplitude_color``` of ```theo_inst```.

In [27]:
new_amp = rand(32767)

theo_inst.amplitude_color = new_amp

LoadError: setfield! immutable struct of type InstrumentTimbre cannot be changed

Which is because the object ```theo_inst``` is stored into a specific location in memory and as was shown in the session, Julia does not allow changes in values stored in a specific reference location.

To alleviate this, we can define a ```mutable struct``` version of the ```struct``` as follows.

In [28]:
mutable struct InstrumentTimbreMut
    amplitude_color::Vector{Float64}
    frequency_color::Vector{Int16}
end

Then defining a new theoretical instrument ```theo_inst_2```,

In [31]:
amp_mut = rand(32767)
freq_mut = 1:32767

theo_inst_2 = InstrumentTimbreMut(amp_mut, freq_mut)

InstrumentTimbreMut([0.2833601866234261, 0.8994534632421041, 0.12544135171167436, 0.3361867422216027, 0.4291921297816532, 0.14743946238982697, 0.1353119324129337, 0.9516480140900014, 0.44669221097244494, 0.8404240440236164  …  0.6691532751216638, 0.5717677436770405, 0.7221382564045389, 0.8083640782547503, 0.14011611984764105, 0.3732783893701821, 0.08085238084668855, 0.14452656776811046, 0.10057546208917323, 0.7371278947635724], Int16[1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  32758, 32759, 32760, 32761, 32762, 32763, 32764, 32765, 32766, 32767])

Then change the values of ```amp_mut```,

In [32]:
amp_mut_2 = 2.0*rand(32767)

theo_inst_2.amplitude_color = amp_mut_2

theo_inst_2

InstrumentTimbreMut([0.851300509777976, 0.004314842143400277, 1.7594899185099497, 0.8018651950194995, 1.2137284771228694, 1.5095772761474953, 0.7076036111689801, 1.171730102074521, 0.4606756083332768, 0.47609594183139414  …  1.8652136835042357, 0.8619203415603334, 1.3532897436150075, 1.9984485551204627, 0.7815376857465353, 0.31331120456844763, 0.26535202116226086, 1.8089615703522655, 1.0425872668307874, 1.5985177811770384], Int16[1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  32758, 32759, 32760, 32761, 32762, 32763, 32764, 32765, 32766, 32767])

Then Julia allows us to change these properties easily. However, this appear to defeat the purpose of ```struct``` since if you would want to do this, then it would be better to define a class instead. From this example, we see that ```struct```s can be used to create lookup tables to make it easier lookup values that you would want to get back to anytime in your code.

There is another type of ```struct``` that you can make in Julia, that is the parametrized ```struct```. With this ```struct```, you may define an object that can take in any type of data (or can be restricted to one supertype), and Julia would be able to determine which object property will store which.

When we try to parametrize our ```InstrumentTimbre``` ```struct```, we see that it won't best to do so since it requires a specific data type. In this case, let us define a different ```struct``` that allows this. Thus, to demonstrate this, let us define another ```struct``` that stores the momentum and energy of an object.

In [2]:
struct MomentumEnergyPar{T<:Number}
    momentum::Vector{T}
    energy::T
end

Doing so makes the most out of parametrized ```struct```s as it defines a ```struct``` with object properties with abstract data types, thus when we instantiate it, we immediately see which variable data type will go to which object property.

In [4]:
particle_momentum = 5.0*rand(2)
particle_energy = 3.25

particle_momenergy = MomentumEnergyPar(particle_momentum, particle_energy)

MomentumEnergyPar{Float64}([2.962367748310063, 1.8533031354767782], 3.25)

# KR3 Type Inferencing in Julia

Like in Python, Julia can immediately infer the data type you are using depending on the expression or values you input in your expressions. An immediate example would be as follows.

In [5]:
[x for x in 1:5]

5-element Vector{Int64}:
 1
 2
 3
 4
 5

In [6]:
[1/x for x in 1:5]

5-element Vector{Float64}:
 1.0
 0.5
 0.3333333333333333
 0.25
 0.2

So notice that while we were inputting integers in our generator expressions, Julia was immediately able to infer the data type we are expected to get, that is for ```x``` we will expect an integer, while for ```1/x``` we will expect a floating point number.

# KR4 Inherent Type-Stability in Julia

